In [14]:
##training
import os,pickle
import pandas as pd
from utils.DataProcess import Start
from utils.ClooingTowerOptimize import CT_opt

opt = CT_opt()
opt.df = Start('HL_14')

opt.train()
with open(os.path.join('model','regr.pkl'), 'wb') as f:
    pickle.dump(opt, f)

冷卻水塔 MAE: 15.39
冰水主機 MAE: 36.27
冷卻水泵 MAE: 9.92


In [2]:
# eval plot
import os,pickle
import numpy as np
import plotly.graph_objects as go
from tqdm import tqdm
from utils.DataProcess import Start
from utils.ClooingTowerOptimize import CT_opt
with open(os.path.join('model','regr.pkl'), 'rb') as p:
    opt = pickle.load(p)


vs = []
for c in tqdm(range(1000)):
    try:
        opt.predict(c)
        v,fig = opt.plot(21,29)
        vs.append(v)
    except:
        pass


fig = go.Figure()
fig.add_traces(go.Scatter(x=opt.df.Datetime[-1000:][::-1],y=opt.df.condenser_supply_temp[-1000:][::-1],mode='lines',name='Manual'))
fig.add_traces(go.Scatter(x=opt.df.Datetime[-1000:][::-1],y=np.array(vs),mode='lines',name='AI'))
fig.add_traces(go.Scatter(x=opt.df.Datetime[-1000:][::-1],y=opt.df.Wet_bulb_temp[-1000:][::-1],mode='lines',name='Wet bulb'))

100%|██████████| 1000/1000 [00:19<00:00, 50.90it/s]


In [22]:
import plotly.express as px
df = opt.df.copy()
df = df[df.CT_Total_KW>0]
px.scatter(data_frame=df,x='CT_eff',y='CT_Total_KW'
,color='Wet_bulb_temp'
)

In [24]:
opt.df.Wet_bulb_temp.plot()

In [15]:
a = opt.predict(1)
a,b = opt.plot(21,29)
b

In [25]:
import plotly.express as px
df = opt.df.copy()
df = df[df.Wet_bulb_temp>0]
px.scatter(
    data_frame=df,
    x='Approach',
    y='CT_Total_KW',
    color='Wet_bulb_temp'
)